<a href="https://colab.research.google.com/github/bf777/MesoNet/blob/master/mesonet_demo_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MesoNet
Welcome to MesoNet, a toolbox for segmenting mesoscale calcium images! This notebook will take you through all the steps needed to process your own calcium imaging dataset using our models.

In [1]:
# Use tensorflow 1.x (supported by DeepLabCut)
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

TensorFlow 1.x selected.
1.15.2


In [1]:
# Clone MesoNet repository
import os
from getpass import getpass
user = getpass('GitHub user')
password = getpass('GitHub password')
os.environ['GITHUB_AUTH'] = user + ':' + password
!git clone -s https://$GITHUB_AUTH@github.com/bf777/MesoNet.git

GitHub user··········
GitHub password··········
Cloning into 'MesoNet'...
remote: Enumerating objects: 250, done.
remote: Counting objects: 100% (250/250), done.
remote: Compressing objects: 100% (179/179), done.
remote: Total 797 (delta 118), reused 193 (delta 67), pack-reused 547
Receiving objects: 100% (797/797), 5.05 MiB | 33.34 MiB/s, done.
Resolving deltas: 100% (502/502), done.


## Prepare inputs

In [ ]:
!mkdir mesonet_inputs/

Now, we will copy sample data (11 images) from the git repository to the inputs folder. If, instead, you would like to upload your own images create a folder inside `mesonet_inputs` and put your images inside that folder; then, skip the next cell. 

In [14]:
# If you have your own input data that you've put into a subfolder under
# mesonet_inputs, don't run this cell.
%cp -r MesoNet/tests/test_input/ mesonet_inputs/

In [ ]:
!pip install deeplabcut

Because of how DeepLabCut operates, you now need to restart your runtime (under the Runtime menu).

In [3]:
%cd MesoNet/

/content/MesoNet


In [4]:
# Utility to install mesonet package and associated requirements (will be replaced with pip later)
!pip install matplotlib==3.1.3
!python setup.py install

# pip install mesonet

running install
running bdist_egg
running egg_info
writing mesonet.egg-info/PKG-INFO
writing dependency_links to mesonet.egg-info/dependency_links.txt
writing requirements to mesonet.egg-info/requires.txt
writing top-level names to mesonet.egg-info/top_level.txt
writing manifest file 'mesonet.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/mesonet
copying build/lib/mesonet/gui_start.py -> build/bdist.linux-x86_64/egg/mesonet
copying build/lib/mesonet/mask_functions.py -> build/bdist.linux-x86_64/egg/mesonet
copying build/lib/mesonet/train_model.py -> build/bdist.linux-x86_64/egg/mesonet
copying build/lib/mesonet/__init__.py -> build/bdist.linux-x86_64/egg/mesonet
copying build/lib/mesonet/atlas_brain_matching.py -> build/bdist.linux-x86_64/egg/mesonet
copying build/lib/mesonet/data.py -> build/bdist.linux-x86_64/egg/mesonet
copying build/lib/mes

We will now pull information from the OSF repository containing the DeepLabCut and U-Net models for our code.

In [2]:
!pip install osfclient

In [7]:
!osf -p svztu -u brandon.forys@alumni.ubc.ca fetch Landmark_estimation_model/atlas-DongshengXiao-2020-08-03.zip mesonet_inputs/atlas-DongshengXiao-2020-08-03.zip

Please input your password: 
100% 643M/643M [00:05<00:00, 112Mbytes/s]


In [ ]:
!unzip mesonet_inputs/atlas-DongshengXiao-2020-08-03.zip

In [8]:
!osf -p svztu -u brandon.forys@alumni.ubc.ca fetch U-Net_model/DongshengXiao_bundary.hdf5 MesoNet/mesonet/models/DongshengXiao_bundary.hdf5

Please input your password: 
100% 373M/373M [00:03<00:00, 114Mbytes/s]


Now, input the information about your input and output images, as well as the U-Net and DeepLabCut models that you would like to use. The default values will use the test data that we've included in the MesoNet git repository (in `MesoNet/mesonet/tests/test_input`). If you're using your own input data, replace `input_filename` below with the name of the folder in `mesonet_inputs` containing your input data.

In [5]:
#@title Input information for the model
input_filename = 'test_input'  #@param {type: "string"}
output_filename = 'mesonet_outputs'  #@param {type: "string"}
model_name = 'DongshengXiao_bundary.hdf5' #@param {type: "string"}
dlc_model_name = 'atlas-DongshengXiao-2020-08-03'  #@param {type: "string"}

In [7]:
!mkdir /content/mesonet_outputs

mkdir: cannot create directory ‘/content/mesonet_outputs’: File exists


In [26]:
%cd /content/

/content


In [10]:
import os
input_file = os.path.join('/content','mesonet_inputs', input_filename)
output_file = os.path.join('/content', output_filename)
model = os.path.join('MesoNet', 'mesonet', 'models', model_name)
dlc_config = os.path.join(dlc_model_name, 'config.yaml')

Now that we've told Colab where to find the input and output folders, let's define the configuration file!

In [11]:
# We need to make sure that DeepLabCut doesn't run with a GUI (which isn't
# supported in Colab).
os.environ["DLClight"]="True"

# Import mesonet and define the configuration file according to your chosen
# inputs
import mesonet
config_file = mesonet.config_project(input_file, output_file, 'test',
                                     mat_save=False, config=dlc_config, 
                                     model=model)

DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)



Using TensorFlow backend.


The config file (by default in `mesonet_outputs`) contains information about how MesoNet will run. We'll be using this config file as an input to the last two functions needed to run MesoNet.

In [12]:
config_file

'/content/mesonet_outputs/mesonet_test_config.yaml'

Next, we'll use the U-net model to identify the outer edges of the cortex:

In [13]:
mesonet.predict_regions(config_file)

/content/MesoNet/mesonet/models/DongshengXiao_bundary.hdf5
/content/mesonet_inputs/test_input
12/12 [==============================] - 8s 647ms/step


In [24]:
os.environ["MESONET_GIT"]='/content/MesoNet/mesonet/'

Lastly, we will identify and use cortical landmarks to align the atlas to the brain:

In [27]:
mesonet.predict_dlc(config_file)

12
/content/mesonet_inputs/test_input/0.png
/content/mesonet_inputs/test_input/1.png
/content/mesonet_inputs/test_input/2.png
/content/mesonet_inputs/test_input/3.png
/content/mesonet_inputs/test_input/4.png
/content/mesonet_inputs/test_input/5.png
/content/mesonet_inputs/test_input/6.png
/content/mesonet_inputs/test_input/7.png
/content/mesonet_inputs/test_input/8.png
/content/mesonet_inputs/test_input/9.png
/content/mesonet_inputs/test_input/10.png
/content/mesonet_inputs/test_input/11.png
Using snapshot-1030000 for model atlas-DongshengXiao-2020-08-03/dlc-models/iteration-2/atlasAug3-trainset95shuffle1
Initializing ResNet
Analyzing all the videos in the directory...
Starting to analyze %  /content/mesonet_outputs/dlc_output/tmp_video.mp4
/content/mesonet_outputs/dlc_output  already exists!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the 

Congratulations, you're all done! You can now check the outputs in the `mesonet_outputs` folder. The segmented brain data can eb found in `mesonet_outputs/output_mask`!